<a href="https://colab.research.google.com/github/KhosrojerdiA/NLP/blob/main/Topic_Modeling_with_Latent_dirichlet_allocation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data
https://github.com/franciscadias/data/blob/master/abcnews-date-text.csv

In [ ]:
import pandas as pd
data = pd.read_csv('abcnews-date-text.csv')
# We only need the Headlines text column from the data
data_text = data[:300000][['headline_text']];
data_text['index'] = data_text.index

documents = data_text

In [ ]:
documents[:5]

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


#Data Preprocessing

In [ ]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

In [ ]:
stemmer = SnowballStemmer("english")

import nltk
nltk.download('wordnet')

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            # TODO: Apply lemmatize_stemming on the token, then add to the results list
            result.append(lemmatize_stemming(token))
    return result

[nltk_data] Downloading package wordnet to /root/nltk_data...


##For one document

In [ ]:
document_num = 4310
doc_sample = documents[documents['index'] == document_num].values[0][0]

print("Original document: ")
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(doc_sample))

Original document: 
['rain', 'helps', 'dampen', 'bushfires']


Tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfir']


##To all

In [ ]:
processed_docs = documents['headline_text'].map(preprocess)

#BOW

In [ ]:
'''
Create a dictionary from 'processed_docs' containing the number of times a word appears
in the training set using gensim.corpora.Dictionary and call it 'dictionary'
'''
dictionary = gensim.corpora.Dictionary(processed_docs)

'''
Checking dictionary created
'''
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [ ]:
'''
Remove very rare and very common words:

- words appearing less than 15 times
- words appearing in more than 10% of all documents
'''

dictionary.filter_extremes(no_below=15, no_above=0.1, keep_n=100000)

In [ ]:
'''
Create the Bag-of-words model for each document i.e for each document we create a dictionary reporting how many
words and how many times those words appear. Save this to 'bow_corpus'
'''
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

##Check for one document

In [ ]:
'''
Checking Bag of Words corpus for our sample document --> (token_id, token_count)
'''
bow_corpus[document_num] #4310

[(71, 1), (107, 1), (462, 1), (3530, 1)]

#TFIDF

In [ ]:
from gensim import corpora, models
#tfidf = # TODO
tfidf = models.TfidfModel(bow_corpus)

'''
Apply transformation to the entire corpus and call it 'corpus_tfidf'
'''
#corpus_tfidf = # TODO
corpus_tfidf = tfidf[bow_corpus]

In [ ]:
'''
Preview TF-IDF scores for our first document --> --> (token_id, tfidf score)
'''
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5959813347777092),
 (1, 0.39204529549491984),
 (2, 0.48531419274988147),
 (3, 0.5055461098578569)]


#LDA with BOW

In [ ]:
lda_model = gensim.models.LdaMulticore(bow_corpus,
                                       num_topics=10,
                                       id2word = dictionary,
                                       passes = 2,
                                       workers=2)

In [ ]:
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic))
    print("\n")

Topic: 0 
Words: 0.021*"test" + 0.020*"lead" + 0.018*"world" + 0.017*"win" + 0.016*"take" + 0.012*"aussi" + 0.012*"action" + 0.011*"strike" + 0.010*"clash" + 0.010*"blue"


Topic: 1 
Words: 0.026*"boost" + 0.023*"health" + 0.020*"servic" + 0.016*"work" + 0.016*"opposit" + 0.012*"drug" + 0.012*"call" + 0.011*"farm" + 0.011*"hear" + 0.011*"timor"


Topic: 2 
Words: 0.018*"home" + 0.017*"return" + 0.016*"blaze" + 0.016*"concern" + 0.015*"nuclear" + 0.014*"hold" + 0.013*"delay" + 0.013*"storm" + 0.012*"firefight" + 0.011*"power"


Topic: 3 
Words: 0.078*"polic" + 0.035*"crash" + 0.026*"investig" + 0.025*"miss" + 0.020*"australia" + 0.018*"open" + 0.017*"search" + 0.016*"die" + 0.016*"probe" + 0.015*"driver"


Topic: 4 
Words: 0.052*"water" + 0.022*"hospit" + 0.016*"resid" + 0.013*"farmer" + 0.012*"drought" + 0.012*"plan" + 0.012*"rain" + 0.012*"park" + 0.012*"expect" + 0.009*"flood"


Topic: 5 
Words: 0.042*"charg" + 0.033*"face" + 0.029*"court" + 0.021*"murder" + 0.018*"polic" + 0.017*"re

#LDA with TFIDF

In [ ]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf,
                                             num_topics=10,
                                             id2word = dictionary,
                                             passes = 2,
                                             workers=4)

In [ ]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print("Topic: {} Word: {}".format(idx, topic))
    print("\n")

Topic: 0 Word: 0.012*"search" + 0.012*"miss" + 0.006*"bodi" + 0.006*"upgrad" + 0.006*"qanta" + 0.005*"afghanistan" + 0.005*"hobart" + 0.005*"growth" + 0.005*"polic" + 0.005*"continu"


Topic: 1 Word: 0.006*"bail" + 0.006*"lebanon" + 0.006*"bali" + 0.006*"light" + 0.005*"plan" + 0.005*"govt" + 0.005*"prison" + 0.005*"revamp" + 0.005*"compo" + 0.005*"council"


Topic: 2 Word: 0.008*"water" + 0.007*"england" + 0.007*"rudd" + 0.006*"restrict" + 0.006*"open" + 0.006*"final" + 0.005*"suppli" + 0.005*"socceroo" + 0.004*"lion" + 0.004*"central"


Topic: 3 Word: 0.007*"illeg" + 0.006*"name" + 0.005*"blue" + 0.005*"test" + 0.005*"pont" + 0.005*"aussi" + 0.005*"pipelin" + 0.005*"elder" + 0.005*"wallabi" + 0.005*"australia"


Topic: 4 Word: 0.022*"polic" + 0.020*"charg" + 0.015*"kill" + 0.014*"murder" + 0.013*"court" + 0.010*"arrest" + 0.010*"jail" + 0.010*"attack" + 0.010*"face" + 0.010*"bomb"


Topic: 5 Word: 0.007*"water" + 0.006*"murray" + 0.006*"bird" + 0.006*"clean" + 0.005*"river" + 0.005*"

#Score

In [ ]:
processed_docs[4310]

['rain', 'help', 'dampen', 'bushfir']

In [ ]:
# Our test document is document number 4310
for index, score in sorted(lda_model[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.48278704285621643	 
Topic: 0.052*"water" + 0.022*"hospit" + 0.016*"resid" + 0.013*"farmer" + 0.012*"drought" + 0.012*"plan" + 0.012*"rain" + 0.012*"park" + 0.012*"expect" + 0.009*"flood"

Score: 0.2200203835964203	 
Topic: 0.018*"home" + 0.017*"return" + 0.016*"blaze" + 0.016*"concern" + 0.015*"nuclear" + 0.014*"hold" + 0.013*"delay" + 0.013*"storm" + 0.012*"firefight" + 0.011*"power"

Score: 0.1571703553199768	 
Topic: 0.071*"govt" + 0.042*"urg" + 0.024*"fund" + 0.019*"help" + 0.015*"school" + 0.013*"accus" + 0.010*"appeal" + 0.010*"decis" + 0.010*"time" + 0.010*"plan"

Score: 0.020003188401460648	 
Topic: 0.026*"boost" + 0.023*"health" + 0.020*"servic" + 0.016*"work" + 0.016*"opposit" + 0.012*"drug" + 0.012*"call" + 0.011*"farm" + 0.011*"hear" + 0.011*"timor"

Score: 0.020003177225589752	 
Topic: 0.078*"polic" + 0.035*"crash" + 0.026*"investig" + 0.025*"miss" + 0.020*"australia" + 0.018*"open" + 0.017*"search" + 0.016*"die" + 0.016*"probe" + 0.015*"driver"

Score: 0.0200031

In [ ]:
for index, score in sorted(lda_model_tfidf[bow_corpus[document_num]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.5864159464836121	 
Topic: 0.009*"break" + 0.008*"drought" + 0.008*"indonesian" + 0.007*"gold" + 0.007*"hick" + 0.007*"hill" + 0.006*"climat" + 0.006*"violenc" + 0.006*"coast" + 0.005*"relief"

Score: 0.25354084372520447	 
Topic: 0.007*"water" + 0.006*"murray" + 0.006*"bird" + 0.006*"clean" + 0.005*"river" + 0.005*"petrol" + 0.005*"govt" + 0.005*"dump" + 0.005*"wast" + 0.005*"plan"

Score: 0.020008277148008347	 
Topic: 0.022*"crash" + 0.012*"die" + 0.011*"blaze" + 0.011*"road" + 0.009*"firefight" + 0.009*"polic" + 0.009*"accid" + 0.007*"flood" + 0.007*"fatal" + 0.007*"investig"

Score: 0.020006973296403885	 
Topic: 0.008*"water" + 0.007*"england" + 0.007*"rudd" + 0.006*"restrict" + 0.006*"open" + 0.006*"final" + 0.005*"suppli" + 0.005*"socceroo" + 0.004*"lion" + 0.004*"central"

Score: 0.020005488768219948	 
Topic: 0.014*"govt" + 0.010*"urg" + 0.009*"plan" + 0.009*"health" + 0.009*"fund" + 0.008*"opposit" + 0.007*"labor" + 0.007*"council" + 0.007*"group" + 0.007*"indigen"

Sco

#Testing model on unseen document

In [ ]:
unseen_document = "My favorite sports activities are running and swimming."

bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.2203076332807541	 Topic: 0.021*"test" + 0.020*"lead" + 0.018*"world" + 0.017*"win" + 0.016*"take"
Score: 0.21998444199562073	 Topic: 0.042*"charg" + 0.033*"face" + 0.029*"court" + 0.021*"murder" + 0.018*"polic"
Score: 0.21994270384311676	 Topic: 0.026*"boost" + 0.023*"health" + 0.020*"servic" + 0.016*"work" + 0.016*"opposit"
Score: 0.21972128748893738	 Topic: 0.052*"water" + 0.022*"hospit" + 0.016*"resid" + 0.013*"farmer" + 0.012*"drought"
Score: 0.02001159079372883	 Topic: 0.071*"govt" + 0.042*"urg" + 0.024*"fund" + 0.019*"help" + 0.015*"school"
Score: 0.020006462931632996	 Topic: 0.018*"home" + 0.017*"return" + 0.016*"blaze" + 0.016*"concern" + 0.015*"nuclear"
Score: 0.020006462931632996	 Topic: 0.078*"polic" + 0.035*"crash" + 0.026*"investig" + 0.025*"miss" + 0.020*"australia"
Score: 0.020006462931632996	 Topic: 0.031*"kill" + 0.027*"iraq" + 0.027*"attack" + 0.016*"howard" + 0.015*"year"
Score: 0.020006462931632996	 Topic: 0.047*"council" + 0.030*"plan" + 0.023*"chang" + 0.